In [48]:
import pandas as pd
import time
import json
from web3 import Web3, HTTPProvider, IPCProvider
import pymongo
import os
import pdb
import tqdm
import web3
import numpy as np

Directory = ''

DB_NAME = "Ethereum_Blockchain"
NewCollection = 'TransactionDataByBlock'

#call database for collection with raw transactions. Have  to launch db first
db = pymongo.MongoClient()[DB_NAME][NewCollection]

#cursor creation

cursor = db.find({"number":{"$gt": 5250000}})

#iterate to put each row into list
#put list of dicts into dataframe

def docUnwrapper(cursor):
    doclist = []
    count = 0
    for doc in cursor:
        doclist.append(doc)
    docframe = pd.DataFrame(doclist)
    
    return docframe

frame = docUnwrapper(cursor)

In [49]:
#prepare frames for groupby. Product of gas + gas price cols

frame['gaspaid'] = frame['gas']*frame['gasprice']
cols = ['_id', 'data', 'from','to', 'gas','gasprice', 'gaspaid','value']
frame = frame[cols]

ContractCallsFrame = frame[frame['data'] != '0x']



In [50]:
#groupby from & to and perform agg functions. Fillna for NA's
def TransactionCountSummary(frame):
    aggframe = frame.groupby('from').agg({
            'to': lambda x: x.nunique(),
            'value':'count',
        })

    aggframe2 = frame.groupby('to').agg({
            'from': lambda x: x.nunique(),
            'value':'count',
        })

    #merge from & to frames and calculate total numbers

    aggframe = aggframe.reset_index()
    aggframe2 = aggframe2.reset_index()

    aggframe = aggframe.rename(columns = {'from':'id', 'to':'uniqueTo', 'value':'transactioncountFrom'})
    aggframe2 = aggframe2.rename(columns = {'to':'id', 'from':'uniqueFrom','value':'transactioncountTo'})

    newframe = pd.merge(aggframe, aggframe2, how = 'outer', on = 'id', suffixes = ['_from','_to']).fillna(0)
    newframe['uniqueTotalAddresses'] = newframe['uniqueTo'] + newframe['uniqueFrom']
    newframe['total'] = newframe['transactioncountFrom'] + newframe['transactioncountTo']
    return newframe

#Create one frame for all transactions and another frame for contract calls
AllTransFrame = TransactionCountSummary(frame)
ContractCalls = TransactionCountSummary(ContractCallsFrame)

analysisframe = pd.merge(AllTransFrame, ContractCalls, how = 'left', on = 'id', suffixes = ['_alltrans','_contractsIO']).fillna(0)

#Combine with mapping table of known addresses

RefDirectory = 'C:\\Users\\haoka\\Documents'
KnownAddressList = 'Known_OwnerAddress_Pairs.xlsx'

os.chdir(RefDirectory)
AddressList = pd.read_excel(KnownAddressList)

AddressList = AddressList.rename(columns = {'Address':'id'})

NamedAnalysisFrame = pd.merge(analysisframe, AddressList, how = 'left', on = 'id')

#Save to file. Filter down top addresses as necessary

NamedAnalysisFrameFiltered = NamedAnalysisFrame[NamedAnalysisFrame['total_alltrans'] > 4]
NamedAnalysisFrameFiltered.to_excel('TransactionFingerprintingAnalysis_5250000-5299999.xlsx')

,id,uniqueTo_alltrans,transactioncountFrom_alltrans,uniqueFrom_alltrans,transactioncountTo_alltrans,uniqueTotalAddresses_alltrans,total_alltrans,uniqueTo_contractsIO,transactioncountFrom_contractsIO,uniqueFrom_contractsIO,transactioncountTo_contractsIO,uniqueTotalAddresses_contractsIO,total_contractsIO,OwnerType,Owner
0,0x00000002f43b5C2aC448f3DBE4F316FA5160d82A,1.0,2.0,0.0,0.0,1.0,2.0,1.0,2.0,0.0,0.0,1.0,2.0,NaN,NaN
1,0x000000043Dc3052d771845a71Efc05B67F40aBb4,1.0,2.0,2.0,2.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,0x0000000484f2217f1A64Eb6d24b5CEe446FaEAe5,9.0,17.0,0.0,0.0,9.0,17.0,7.0,14.0,0.0,0.0,7.0,14.0,NaN,NaN
3,0x000000060d5F3AF420ba8CE05d7B6151b62931e6,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,NaN,NaN
4,0x000000571faa37B87eA4354E0d5428296362c285,4.0,4.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [51]:
#Filter big frame for top 500 account transactions

Top500 = NamedAnalysisFrame.nlargest(500, 'total_alltrans').reset_index()
cols = ['id', 'OwnerType','Owner'] 
Top500LessCols = Top500[cols]

#Function for separating out Layer 1 addresses and transactions

#Still need to clean up col names and suffixes so that frames combine correctly.

def NextLayerFilter(Layer0Frame, MasterFrame, ID = 'id'):
    """Takes a 'Layer0' frame and filters for all transactions 
    belonging to accounts that interacted with it. Column name for
    ID in layer0frame taken as karg. Returns filtered raw transaction list."""
    
    #reducesize of masterframe for creating filterframes
    neededcols = ['from', 'to']
    
    MasterFrameCut = MasterFrame[neededcols]
    
    #Create 'from' and 'to' filter frames of unique addresses transacted with.
    #Uses inner join instead of isin for speed optimization purposes.
    Layer0FrameFrom = Layer0Frame.rename(columns = {ID:'from'})
    fromframe = MasterFrameCut.merge(Layer0FrameFrom, how = 'inner', on = 'from', suffixes= ['_x', '_y'])
    fromframecut = fromframe.drop_duplicates(subset = ['from','to'])
    
    Layer0FrameTo = Layer0Frame.rename(columns = {ID:'to'})
    toframe = MasterFrameCut.merge(Layer0FrameTo, how = 'inner', on = 'to', suffixes= ['_x', '_y'])
    toframecut = toframe.drop_duplicates(subset = ['from','to'])
    
    
    #Make list of unique addresses from lists of 'from' and 'to' transaction addresses. 
    #If frames too big to hold in memory, implement without copying masterframe below.
    
    MasterFrameAddr1 = MasterFrame.rename(columns = {'from': 'id'})
    MasterFrameAddr2 = MasterFrame.rename(columns = {'to': 'id'})
    
    fromframecut = fromframecut.rename(columns = {'to':'id'})
    
    MasterFrameAddrTo1 = MasterFrameAddr1.merge(fromframecut, how = 'inner', on='id', suffixes = ['_x','_y'])
    MasterFrameAddrTo2 = MasterFrameAddr2.merge(fromframecut, how = 'inner', on='id', suffixes = ['_x','_y'])
    
    
    toAddressTransactions = pd.concat([MasterFrameAddrTo1, MasterFrameAddrTo2])
    #Final frame of 'toAddressTransactions' should represent all the transactions for addresses that were sent
    #transactions from the top 500 transacting accounts. (Top 500 => These accounts)
    
    toframecut = toframecut.rename(columns = {'from':'id'})
    
    MasterFrameAddrTo1 = MasterFrameAddr1.merge(toframecut, how = 'inner', on='id', suffixes = ['_x','_y'])
    MasterFrameAddrTo2 = MasterFrameAddr2.merge(toframecut, how = 'inner', on='id', suffixes = ['_x','_y'])
    
    
    fromAddressTransactions = pd.concat([MasterFrameAddrTo1, MasterFrameAddrTo2])
    
    #Final frame of 'toAddressTransactions' should represent all the transactions for addresses that sent
    #transactions to the top 500 transacting accounts. (These accounts => Top 500)
    
    rawtransactions = pd.concat([fromAddressTransactions, toAddressTransactions])
    
    return rawtransactions


#Layer 1 summaries. 

    
    
    
    
    
    
    


In [47]:
#testing cell

cursortest = db.find({"number":{"$gt": 5200000, "$lt": 5250000}, "from":'0x876EabF441B2EE5B5b0554Fd502a8E0600950cFa'})
count = 0
for doc in cursortest:
    print(doc)
    count +=1
    if count > 10:
        break

{'_id': ObjectId('5ac663de5e18d130b8b132f7'), 'number': 5200003, 'timestamp': 1520240839, 'from': '0x876EabF441B2EE5B5b0554Fd502a8E0600950cFa', 'to': '0x86Fa049857E0209aa7D9e616F7eb3b3B78ECfdb0', 'value': 0, 'data': '0xa9059cbb0000000000000000000000001535e5a609a5c96857d58a55effaad1f173d61640000000000000000000000000000000000000000000000db52682f10bc018000', 'gas': 90000, 'gasprice': 30000000000}
{'_id': ObjectId('5ac663e45e18d130b8b139be'), 'number': 5200012, 'timestamp': 1520241033, 'from': '0x876EabF441B2EE5B5b0554Fd502a8E0600950cFa', 'to': '0xd26114cd6EE289AccF82350c8d8487fedB8A0C07', 'value': 0, 'data': '0xa9059cbb000000000000000000000000d706e41950231de0d6c5df2ed1b734c01fa42fcc0000000000000000000000000000000000000000000000104163ccce14fdc000', 'gas': 90000, 'gasprice': 30000000000}
{'_id': ObjectId('5ac663e55e18d130b8b13b21'), 'number': 5200013, 'timestamp': 1520241061, 'from': '0x876EabF441B2EE5B5b0554Fd502a8E0600950cFa', 'to': '0x86Fa049857E0209aa7D9e616F7eb3b3B78ECfdb0', 'value': 0

In [10]:
#testing cell

count = 0
for doc in cursor:
    print(doc['_id'])
    count += 1
    if count > 3:
        break

5ac663da5e18d130b8b12f33
5ac663da5e18d130b8b12f34
5ac663da5e18d130b8b12f35
5ac663da5e18d130b8b12f36
